# Introduction
Most of the code found here was originally taken from the [PyTorch Quickstart](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html) but was and will be adapted over time to better suit me.

## Imports

In [1]:
# Imports 
import torch
import torchvision

## Constructing a model

In [2]:
# Define a Model
class Net(torch.nn.Module):
    """ Constructor """
    def __init__(self):
        super(Net, self).__init__()
        
        self.stack = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(784, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 10),
            torch.nn.ReLU()
        )
        
    """
        Define forward-pass
    """
    def forward(self, x):
        return self.stack(x)

In [3]:
# Check if CUDA drivers are available and change the device type accordingly
device_type = "cuda" if torch.cuda.is_available() else "cpu"
model = Net().to(device_type)

print(model)

Net(
  (stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=10, bias=True)
    (6): ReLU()
  )
)


/home/tobias/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


## Loading data
The following is an example of how to load datasets that are built-in in either TorchText, TorchVision or TorchAudio.

In [4]:
# Download training data from open datasets.
train_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)

# Download test data from open datasets.
test_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor(),
)

We also have to create data loaders so that the datasets can be used by pytorch.

In [5]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size)

## Training a model
We start by defining training objectives and optimizers.

In [6]:
# Define a loss function
loss = torch.nn.CrossEntropyLoss() # Mean squared error

# Define an optimizer
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # Stochastic Gradient Descent

Next we define a training loop.

In [7]:
def epoch(loader, model, loss_fn, optimizer):
    dataset_size = len(loader.dataset)
    
    # Iterate through all batches
    for batch_nr, (X, y) in enumerate(loader):
        # .to( ) is a device or datatype conversion 
        X = X.to(device_type)
        y = y.to(device_type)
        
        prediction = model(X) # Forward pass
        loss = loss_fn(prediction, y) # Compute loss
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Give some feedback on the terminal
        if batch_nr % 100 == 0: # Plot loss on every 100th batch
            loss = loss.item()
            print(f"Batch Number: {batch_nr} => loss: {loss}")

And an evaluation loop

In [8]:
def test(loader, model, loss_fn):
    dataset_size = len(loader.dataset)
    model.eval() # Set the model to evaluation mode
    
    test_loss = 0.0 # Keep track of calculated loss
    correct = 0.0 # Keep track of how many predictions where correct
    
    # The following code is executed so that no gradient is calculated
    with torch.no_grad():
        for X,y in loader:
            X = X.to(device)
            y = y.to(device)
            
            # Make predictions
            prediction = model(X)
            
            # Calculate metrics
            test_loss += loss_fn(prediction,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # Evaluate 1hot

    # Normalize metrics
    test_loss /= dataset_size
    correct /= dataset_size
            

We can use this for a full set of training

In [9]:
epochs = 10
for t in range(epochs):
    # Do some training and testing here
    pass

## Loading and Saving models

In [10]:
filename = "some_name.pth"

In [11]:
torch.save(model.state_dict(), filename)

In [12]:
model = Net()
model.load_state_dict(torch.load(filename))

<All keys matched successfully>

## Sources:
- **[PyTorch Quickstart Tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)**
